I have currently applied for citizenship. It's a long process without a set schedule. So rather than waste time going to the website, enter my case number, navigate to the status page, and check if there is any change, I decided to automate the steps. So I wrote a cron job to check whether there was any changes to my status and if so to shoot me an e-mail.
Here I leverage [SELENIUM](http://www.seleniumhq.org/projects/webdriver/), [BEAUTIFULSOUP](https://www.crummy.com/software/BeautifulSoup/), and [SMTPLIB](https://docs.python.org/3/library/smtplib.html) and [PYTHON-CRONTAB](https://pypi.python.org/pypi/python-crontab). 

Note that building my automaton requires scouting the pages it needs to navigate to identify the specific elements it needs to interact with; in this case only a handful.

I use python 3.6.

First the necessary imports...

In [1]:
from selenium import webdriver
import pathlib
import os
from bs4 import BeautifulSoup as Soup
from shutil import which

The last step I went through in my process is an interview with a case officer. Thus the phrase that if altered on my status page should indicate a a change in my status is "we scheduled an interview". Let's first store that in a token...

In [2]:
token = 'we scheduled an interview'

Here, I use [phantomjs](http://phantomjs.org/) to avoid the browser window. **phantompath** gives the location of phantomjs on my computer. If you prefer chromedriver [Miguel Grinberg (the Flask guy) has a recipe](https://blog.miguelgrinberg.com/post/using-headless-chrome-with-selenium) for the cost of a couple extra lines.

In [3]:
phantompath = which('phantomjs')
browser = webdriver.PhantomJS(executable_path=phantompath)

Navigate to the USCIS landing page...

In [4]:
browser.get('https://egov.uscis.gov/casestatus/landing.do')

... find the input box to enter my case number, stored in the variable case_num...

In [5]:
caseNo = browser.find_element_by_id('receipt_number')
caseNo.send_keys(case_num)

... find the button to initiate the lookup and click...

In [6]:
srchButtn = browser.find_element_by_name('initCaseSearch')
srchButtn.click()

... now we're on the status page, get the page source and parse it with BeautifulSoup

In [7]:
pageSrc = browser.page_source
pageSoup = Soup(pageSrc, 'lxml')

... from inspecting the page source, I know that the text I'm looking for is under tag "p"...

In [8]:
targetSection = pageSoup.find('p')

... and is the first item yielded by the child of targetSection (a generator)...

In [9]:
targeText = list(targetSection.children)[0]

... sampling some of the text returns...

In [10]:
targeText[20:100]

' we scheduled an interview for your Form N-400, Application for Naturalization ,'

... well, the status still reflects the interview, so nothing happens after this and the process goes back to sleep. However, the rest of the code shows the next steps in the case of change. Namely
* craft message including the new status text
* intantiate an SMTP object
* call the SMTP object's sendmail method to send the email from/to the appropriate sender/receiver pair, in my case they are one and the same...

In [13]:
if token not in targeText:
    message = f'USCIS CASE UPDATE: \n {targeText}'
    try:
        smtpObj = smtplib.SMTP('localhost')
        smtpObj.sendmail(from_, to, message)
    except SMTPException:
        print("Problem sending email")

The full source code is available at the end of the page. I have saved in a file named **CheckCaseStatus.py**, which I will provide to my crontab as shown below. First import CronTab and instantiate a cron object.

In [14]:
from crontab import CronTab

myCron = CronTab(user=True)

I'm the only user on my computer so it's easier to specify user=True, which then points to the current user. If needed, an actual username can be specified. Now add running the CheckCaseStatus.py as a new job to the cron instance...

In [15]:
runpath = os.path.join('~/DEV/MyCronJobs/CheckCaseStatus.py')

In [16]:
job = myCron.new(command='python CheckCaseStatus.py')

... set the running schedule - check for a change every 5 days - and save...

In [17]:
job.day.every(5)
myCron.write()

In [ ]:
# CheckCaseStatus.py
from selenium import webdriver
import os
from bs4 import BeautifulSoup as Soup
import smtplib
from shutil import which

def check_uscis_page(caseNum, phantompath):
    """
    opens uscis webpage in the background (phantomjs)
    navigates to case status page (requires case number)
    """
    browser = webdriver.PhantomJS(executable_path=phantompath)
    browser.get('https://egov.uscis.gov/casestatus/landing.do')
    caseNo = browser.find_element_by_id('receipt_number')
    caseNo.send_keys(caseNum)
    srchButtn = browser.find_element_by_name('initCaseSearch')
    srchButtn.click()
    pageSrc = browser.pageSource
    pageSoup = Soup(pageSrc, 'lxml')
    browser.close()
    targetSection = pageSoup.find('p')
    return list(targetSection.children)[0]

def check_update(targeText, token):
    if token in targeText:
        return False
    else:
        return True

def shoot_email(text, from_, to):
    from_ = from_
    to = [to]
    message = f'USCIS CASE UPDATE: \n {text}'
    try:
        smtpObj = smtplib.SMTP('localhost')
        smtpObj.sendmail(from_, to, message)
    except SMTPException:
        print("Problem sending email")

if __name__ == '__main__':
    myEmail = os.getenv('MYGMAIL')
    case = os.getenv('USCIS_CASE_NO')
    phantompath = which('phantomjs')   
    retrievedText = check_uscis_page(case, phantompath)
    if  check_update(retrievedText, 'we scheduled an interview'):
        shoot_email(targeText, myEmail, myEmail) 

In [ ]:
# CronScheduler.py
from crontab import CronTab

myCron = CronTab(user=True)
job = myCron.new(command='python CheckCaseStatus.py')
job.day.every(5)
myCron.write()

Done, and all that in 50 or so lines. The last thing left is to *python CronScheduler.py* to get the program going.  

Happy hacking!